# Práctica 3: Ajuste por Mínimos Cuadrados y Factorización QR
**Problemas Computacionales**\
Alumno: Martínez de la Cruz José Jorge\
Profesor: César Carreón Otañez\
Ayudante:  Jesús Iván Coss Calderón

19. a) Resolver el siguiente problema de mínimos cuadrados usando cualquier método de los vistos en clase.
$$
\begin{pmatrix} 0.16 & 0.10 \\ 0.17 & 0.11 \\ 2.02 & 1.29 \end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \end{pmatrix} ≈ \begin{pmatrix} 0.26 \\ 0.28 \\ 3.31 \end{pmatrix}
$$


In [57]:
import numpy as np

In [55]:
A = np.array([[0.16, 0.10],
                   [0.17, 0.11],
                   [2.02, 1.29]])
b = np.array([0.26, 0.28, 3.31 ])

Lo haremos con ecuaciones normales, para ello primero carguemos el programa de Cholesky.

In [65]:
def SustDelante(L, b):
    """
    Resuelve un sistema de ecuaciones L * x = b, donde L es una matriz triangular inferior.

    Parámetros:
    L (numpy.ndarray): Matriz triangular inferior.
    b (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema L * x = b.
    """
    x = np.zeros_like(b)  # Crea un vector x de ceros del mismo tamaño que b
    n = L.shape[0]  # Obtiene el número de renglones de L
    for i in range(n):  # Recorre las filas de L
        sum = 0.0
        for j in range(i):  # Suma los productos de la parte ya resuelta de x
            sum += L[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (b[i] - sum) / L[i, i]

    return x  # Devuelve la solución


In [66]:
def SustAtras(U, y):
    """
    Resuelve un sistema de ecuaciones U * x = y, donde U es una matriz triangular superior.

    Parámetros:
    U (numpy.ndarray): Matriz triangular superior.
    y (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema U * x = y.
    """
    x = np.zeros_like(y)  # Crea un vector x de ceros del mismo tamaño que y
    n = U.shape[0]  # Obtiene el número de renglones de U
    x[n-1] = y[n-1] / U[n-1, n-1]  # Resuelve el último valor de x
    for i in range(n-2, -1, -1):  # Recorre las filas desde la penúltima hasta la primera
        sum = 0.0
        for j in range(i+1, n):  # Suma los productos de los valores ya resueltos de x
            sum += U[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (y[i] - sum) / U[i, i]

    return x  # Devuelve la solución


In [67]:
import numpy as np

def Cholesky(A):
    """
    Realiza la descomposición de Cholesky de una matriz simétrica y definida positiva.

    La descomposición de Cholesky de una matriz A se expresa como:
    A = L * L^T
    donde:
        - A es la matriz de entrada (simétrica y definida positiva).
        - L es una matriz triangular inferior (con ceros sobre la diagonal).

    Parámetros:
    A (numpy.ndarray): Matriz cuadrada simétrica y definida positiva de tamaño n x n.

    Retorna:
    L (numpy.ndarray): Matriz triangular inferior de tamaño n x n tal que A = L * L^T.

    Excepciones:
    Si la matriz A no es simétrica o definida positiva, este algoritmo puede fallar
    o devolver resultados incorrectos.

    El algoritmo sigue el siguiente procedimiento:
    1. Inicializa una matriz L de ceros de las mismas dimensiones que A.
    2. Calcula los elementos de la matriz triangular inferior L usando la fórmula de Cholesky.
    3. La diagonal de L es calculada con la raíz cuadrada de los elementos de A.
    4. Los elementos fuera de la diagonal se calculan usando las sumas de productos previamente computadas.

    """
    # Determina el tamaño de la matriz A
    n = len(A)

    # Crea una matriz L de ceros del mismo tamaño que A
    L = np.zeros_like(A)

    # Recorre las filas y columnas de A para calcular los elementos de L
    for i in range(n):
        for j in range(i + 1):
            # Si estamos en la diagonal principal
            if i == j:
                # Calculamos la suma de los cuadrados de los elementos previos de la fila
                sum = 0.0
                for k in range(j):
                    sum += L[j][k] * L[j][k]
                # La raíz cuadrada del valor en A[j][j] menos la suma anterior da el valor de L[j][j]
                L[j][j] = np.sqrt(A[j][j] - sum)
            else:
                # Si estamos en la parte inferior de la matriz (por debajo de la diagonal)
                sum = 0.0
                for k in range(j):
                    sum += L[i][k] * L[j][k]
                # Usamos la fórmula para calcular los valores fuera de la diagonal
                L[i][j] = (A[i][j] - sum) / L[j][j]

    return L

Ahora sí, definimos el programa que resolverá mediante ecuaciones cuadradas

In [68]:
def EcNormal(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando el método de los mínimos cuadrados
    con la descomposición de Cholesky de la matriz A^T * A.

    Este método se utiliza cuando el sistema no tiene solución exacta o es sobredeterminado (más ecuaciones que incógnitas).
    En lugar de resolver el sistema directamente, se resuelve el sistema equivalente A^T * A * x = A^T * b,
    que tiene solución en el sentido de los mínimos cuadrados.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones de tamaño m x n (m >= n).
    b (numpy.ndarray): Vector columna de tamaño m, que es el lado derecho del sistema de ecuaciones.

    Retorna:
    xSol (numpy.ndarray): El vector solución x, que es la solución de mínimos cuadrados al sistema A * x = b.

    Procedimiento:
    1. Calculamos el sistema equivalente A^T * A * x = A^T * b.
    2. Aplicamos la descomposición de Cholesky a la matriz A^T * A para obtener una matriz triangular inferior L.
    3. Resolvemos el sistema triangular inferior L * y = A^T * b mediante sustitución hacia adelante (SustDelante).
    4. Resolvemos el sistema triangular superior L^T * x = y mediante sustitución hacia atrás (SustAtras).
    """

    # Calculamos el sistema equivalente A^T * A * x = A^T * b
    AtA = A.T @ A  # Matriz generalizada (A^T * A)
    Atb = A.T @ b  # Vector del sistema equivalente (A^T * b)

    # Descomposición de Cholesky de A^T * A
    L = Cholesky(AtA)  # L es la matriz triangular inferior tal que A^T * A = L * L^T
    Lt = L.T  # La transpuesta de L, es la matriz triangular superior

    # Resolvemos el sistema L * y = A^T * b mediante sustitución hacia adelante
    ySol = SustDelante(L, Atb)

    # Resolvemos el sistema L^T * x = ySol mediante sustitución hacia atrás
    xSol = SustAtras(Lt, ySol)

    return xSol


Implementamos el código

In [69]:
EcNormal(A, b)

array([1., 1.])

Con lo que tenemos que la solución al sistema es el vector $(1, 1)$

b) Resolver el mismo problema con la siguiente perturbación en el vector $b$
$$b=
\begin{pmatrix} 0.27 \\ 0.25 \\ 3.33 \end{pmatrix}
$$

In [71]:
b_gorro = np.array([0.27, 0.25, 3.33])
EcNormal(A, b_gorro)

array([ 7.00888731, -8.39566299])

c) Compara los resultados de los incisos anteriores, ¿se puede explicar la diferencia entre ellos?. Incisos a) y b).
La diferencia entre los dos vectores es grandisima, aunque podría creerse que no es mucho cambio en términos de distancia, comparado con la perturbación del vector solución que fue mínima, el proceso para encontrar la solución fue más complejo pues: el cambio en el vector solución no fue lineal ni creciente, es decir, a cada elemento del vector $b$ oríginal se le aplicó una traslación diferente, esto generó que el vector solución no tuviera un cambio lineal definido y por ende, se ocupabá de una solución en el espacio diferente.